# query optimization

the process of query optimization is very similar to the process of refactoring.

the goal is to make the query more efficient without changing it's behavior.

so ideally you should write TDD-style tests for your queries first before you start optimizing them to avoid any unexpected behavior changes.

in this notebook we will go through some common query optimization techniques based on examples.

In [2]:
!chmod +x ./reset.sh && ./reset.sh > /dev/null
!psql postgres -c "reset all;"

Did not find any relations.
psql: error: connection to server on socket "/tmp/.s.PGSQL.5432" failed: FATAL:  database "sueszli" does not exist
ERROR:  cannot drop the currently open database
ERROR:  current user cannot be dropped
RESET


In [3]:
# from enum import Enum
# import os
# import json

# # execution plan
# def print_plan(query: str) -> None:
#     output = os.popen(f'psql postgres -c "{mode.value} explain (analyze, costs) SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"').read()
#     print(output)

# # benchmark
# def get_exec(mode:JoinMode) -> float:
#     full_query = f"{mode.value} explain (analyze, verbose, costs, settings, buffers, wal, timing, summary, format json) SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t;"
#     output = os.popen(f'psql postgres -c "{full_query}"').read()
#     output = output.splitlines()[5:-2]
#     for i in range(len(output)):
#         if len(output[i]) > 2:
#             output[i] = output[i][:-1]
#     output = "\n".join(output)
#     output = json.loads(output)[0]
#     output = float(output["Execution Time"])
#     return output

# def print_avg_exec(mode:JoinMode, iters:int=10) -> None:
#     vals = []
#     for _ in range(iters):
#         time = get_exec(mode)
#         vals.append(time)
#     print(f"average execution time in {mode.name}-MODE: {sum(vals) / iters:.2f} ms (in {iters} iterations)")